
# YOLO v1 Implemetation

미리 만들 단계에 대해서 설명해보겠음!
일단, 당연하게
1. 데이터를 불러오고 데이터 전처리를 실행할 것임 (image 데이터는 input shape를 맞춰주고 label 데이터 또한 텐서로)
2. 모델 훈련시키기! - 사전훈련된 모델인 densenet 을 써볼 것임! (근데 여기에 어느 수준까지를 다 포함하고 있는지를 모르겠음,,)
3. 레이어 더 얹기가 필요한 거 같은데 이걸 논문이랑 비교하면서 봐야 하는 부분인 것 같음.
4. loss 함수 정의
5. 훈련 세팅하기 -> custom learning rate scheduler 정의하기
6. 모델 컴파일 하기 -> loss랑 optimizer 설정해서!
7. 모델 훈련하기 -> modle.fit 으로 135 에포크 만큼 훈련하기


In [1]:
import tensorflow as tf
import argparse
import xml.etree.ElementTree as ET
import os
import matplotlib.pyplot as plt
%matplotlib inline

## 1. 데이터 전처리

1-1) xml to text

xml로 되어 있는 데이터를 전부 하나하나 txt 파일로 변경해주는 과정!.. 이걸줄 알았으면 xml을 csv파일로 바꿔서 쓰는 걸 알아볼 걸 그랬나?
그런데 일단 이해하는 중이니 하나 씩 다 해보고 효율적인 방법으로 바꾸는 걸로 하자!

In [2]:
#파서 만들기
#ArgumentParser 객체를 생성
parser = argparse.ArgumentParser(description='Build Annotations.')
#인자 추가하기
#ArgumentParser 에 프로그램 인자에 대한 정보 채우기 -> parse_args() 호출하면 default를'..'으로 하는 dir이 생성됨!
parser.add_argument('dir', default='..', help='Annotations.')

#sets 리스트
sets = [('2007', 'train'), ('2007', 'val')]

#class 넘버 지정 
classes_num = {'aeroplane': 0, 'bicycle': 1, 'bird': 2, 'boat': 3, 'bottle': 4, 'bus': 5,
               'car': 6, 'cat': 7, 'chair': 8, 'cow': 9, 'diningtable': 10, 'dog': 11,
               'horse': 12, 'motorbike': 13, 'person': 14, 'pottedplant': 15, 'sheep': 16,
               'sofa': 17, 'train': 18, 'tvmonitor': 19}

#annotation 을 컨버트 해주는 함수 => xml 파일 속 object 태그에 대한 값들을 정리하는중!
def convert_annotation(year, image_id, f):
    in_file = os.path.join('yolo1\VOCdevkit\VOC%s\Annotations/%s.xml' % (year, image_id))
    #파일 가져오기
    tree = ET.parse(in_file)
    #최상위 tag 가져오기
    root = tree.getroot()

    """xml 파일에서 object tag 속 목록!
    <object>
		<name>dog</name>
		<pose>Left</pose>
		<truncated>1</truncated>
		<difficult>0</difficult>
		<bndbox>
			<xmin>48</xmin>
			<ymin>240</ymin>
			<xmax>195</xmax>
			<ymax>371</ymax>
		</bndbox>
	</object>
    """
    for obj in root.iter('object'):
        #difficult 속 값 (이를 text라 부름)을 저장
        difficult = obj.find('difficult').text
        Class = obj.find('name').text
        classes = list(classes_num.keys())
        #Class가 classes 안에 없거나 difficult 의 값이 1이면 
        if Class not in classes or int(difficult) == 1:
            continue
        Class_id = classes.index(Class)
        xmlbox = obj.find('bndbox')
        #b(xmin, ymin, xmax, ymax)임!
        b = (int(xmlbox.find('xmin').text), int(xmlbox.find('ymin').text),
             int(xmlbox.find('xmax').text), int(xmlbox.find('ymax').text))
        #bounding box 정보와 class_id (숫자로 표시된)를 문자열로 써주기
        f.write(' ' + ','.join([str(a) for a in b]) + ',' + str(Class_id))

In [3]:
#여기서! 우리는 test 파일을 안 합쳐주었기에 이미지셋에서 빠져있다!
for year, image_set in sets:
    print(year, image_set)
    with open(os.path.join('yolo1\VOCdevkit\VOC%s\ImageSets\Main\%s.txt'%(year, image_set)),'r') as f:
        image_ids = f.read().strip().split()
    with open(os.path.join("yolo1\VOCdevkit", '%s_%s'%(year, image_set)),'w') as f:
        for image_id in image_ids:
            f.write('yolo1\%s\VOC%s\JPEGImages\%s.jpg' % ("VOCdevkit",year,image_id))
            convert_annotation(year, image_id, f)
            f.write('\n')

2007 train
2007 val


1-2) 인풋과 아웃풋 준비!

Input은 (448,448,3)이고 Output은 (7,7,30) 텐서기에 이들을 준비한다.

In [4]:
import cv2 as cv
import numpy as np

In [5]:
#cv2를 열심히 이용한다.
def Img_read(image_path, label):
    #이미지 불러오기
    image = cv.imread(image_path)
    #기존의 BGR 컬러를 RGB로 바꾸기
    image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
    #이미지의 height, width 가져오고 리사이즈 해주고 
    image_h, image_w = image.shape[0:2]
    image = cv.resize(image, (448, 448))
    #여기선 픽셀범위인 0~255 를 신경망에 넣어줄 input인 0~1.0 으로 변환
    image = image / 255.

    #넘파이 배열로 (7,7,30)에 0을 다 깔아준 label_matrix
    label_matrix = np.zeros([7, 7, 30])
    for l in label:
        l = l.split(',')
        l = np.array(l, dtype=np.int)
        #각각의 좌표 얻기
        xmin = l[0]
        ymin = l[1]
        xmax = l[2]
        ymax = l[3]
        Class = l[4]
        
        x = (xmin + xmax) / 2 / image_w
        y = (ymin + ymax) / 2 / image_h
        #w, h를 0~1.0 사이의 값으로 만들기
        w = (xmax - xmin) / image_w
        h = (ymax - ymin) / image_h
        loc = [7 * x, 7 * y]
        loc_i = int(loc[1])
        loc_j = int(loc[0])
        #x,y 를 0~1.0 의 값으로 만들기
        y = loc[1] - loc_i
        x = loc[0] - loc_j

        if label_matrix[loc_i, loc_j, 24] == 0:
            label_matrix[loc_i, loc_j, Class] = 1
            label_matrix[loc_i, loc_j, 20:24] = [x, y, w, h]
            label_matrix[loc_i, loc_j, 24] = 1

    return image, label_matrix

## 모델 훈련하기

인풋과 아웃풋을 배치로 리턴하는 커스텀 제너레이터 정의하기 

## 그치만 진짜 할 것은 사전훈련된 모델 써보기

densenet을 써볼 생각!

In [6]:
from tensorflow import keras

#데이터를 사용할 수 있게 처리
class My_Custom_Generator(keras.utils.Sequence) :
    
    #클래스의 생성할 때 인자들을 받아옴
    def __init__(self, images, labels, batch_size) :
        self.images = images
        self.labels = labels
        self.batch_size = batch_size
    
    #리턴해주는 길이를 범위로 해 getitem함수에 사용될 index로 반환
    def __len__(self) :
        return (np.ceil(len(self.images) / float(self.batch_size))).astype(np.int)
        #이미지개수를 배치수로 나누어 한 epoch내에 존재할 수 있는 배치수를 길이로 반환
    
    #실질적으로 배치 반환
    #함수 인자 속 index는 range(0,len)사이의 인덱스 차례대로 반환
    #해당 인덱스를 참조하여 해당인덱스에 해당하는 이미지, 마스크를 배치사이즈만큼 불러와 배치 새엇ㅇ
    def __getitem__(self, index) :
        batch_x = self.images[index * self.batch_size : (index+1) * self.batch_size]
        batch_y = self.labels[index * self.batch_size : (index+1) * self.batch_size]

    #train_image와 train_Label 은 리스트로 만들어주기
        image_dataset = []
        label_dataset = []

    #image_path랑 label 데이터 처리
        for i in range(0, len(batch_x)):
            img_path = batch_x[i]
            label = batch_y[i]
            image, label_matrix = Img_read(img_path, label)
            image_dataset.append(image)
            label_dataset.append(label_matrix)
            
        
        return np.array(image_dataset), np.array(label_dataset)

인풋과 아웃풋을 배열로 준비하기

In [7]:
#모든 정보를 배열로 준비하기!

train_datasets = []
val_datasets = []

#아까 생성해준 txt 파일을 불러와서 train, val datasets안에 넣어주기
with open(os.path.join("yolo1\VOCdevkit", '2007_train'), 'r') as f:
    train_datasets = train_datasets + f.readlines()
with open(os.path.join("yolo1\VOCdevkit", '2007_val'), 'r') as f:
    val_datasets = val_datasets + f.readlines()


train_image_dataset = []
train_label_dataset = []

val_image_dataset = []
val_label_dataset = []

for item in train_datasets:
    item = item.replace("\n", "").split(" ")
    train_image_dataset.append(item[0])
    arr = []
    for i in range(1, len(item)):
        arr.append(item[i])
    train_label_dataset.append(arr)

for item in val_datasets:
    item = item.replace("\n", "").split(" ")
    val_image_dataset.append(item[0])
    arr = []
    for i in range(1, len(item)):
        arr.append(item[i])
    val_label_dataset.append(arr)

In [18]:
#배치 사이즈를 정해준다
batch_size = 4

#training 에서와 validation의 batch_generator 만들어주고
my_training_batch_generator = My_Custom_Generator(train_image_dataset, train_label_dataset, batch_size)

my_validation_batch_generator = My_Custom_Generator(val_image_dataset,val_label_dataset, batch_size)

#__getitem__으로 배치 리턴해주기
x_train, y_train = my_training_batch_generator.__getitem__(0)
x_val, y_val = my_training_batch_generator.__getitem__(0)
print(x_train.shape)
print(y_train.shape)

print(x_val.shape)
print(y_val.shape)

(4, 448, 448, 3)
(4, 7, 7, 30)
(4, 448, 448, 3)
(4, 7, 7, 30)


x_train, x_val 의 shape가 (448,448,3)이고 이들이 인풋 이미지가 된다.

## 커스텀 아웃풋 레이어 정의하기

모델로부터의 아웃풋을 reshape 해줘야 하기에 custom Keras Layer를 정의한다.



In [37]:
#원래는 코드였지만 잠시 마크다운으로 바꿔놓겠음! 영원히 마크다운이길 바람.
from tensorflow import keras
import keras.backend as K

#흠.. 이게 잘 이해가 안된다.. 그래서 이걸 아예 없애보겠음! 왜냐면 tensorflow 의 reshape를 구현해준 거 같다고 민규님께서 말씀해주셨다.

#이 사람이 tensorflow에 있는 np.reshape()을 몰라서 안 쓰신 것 같다고 했다.그래서 이 부분을 좀 고쳐보고싶은데..


## 사전훈련된 불러와 사용하기

tensorflow에서 제공하는 DenseNet121를 불러와 훈련시킬 모델의 Backbone으로 사용한다.
https://www.tensorflow.org/api_docs/python/tf/keras/applications/densenet/DenseNet121

input_shape를 x_train, x_val 의 shape인 (448,448,3)에 맞춰주었고 output은 (7,7,30) 형태로 reshape 해주었다.

In [38]:
#yolo 모델 층 쌓기 => 지금 이 부분을 아에 pretrained 된 걸 쓰려 하는거!

nb_boxes=1
grid_w=7
grid_h=7
cell_w=64
cell_h=64
img_w=grid_w*cell_w
img_h=grid_h*cell_h


In [39]:
conv_base = tf.keras.applications.densenet.DenseNet121(
    include_top=False, weights='imagenet',
    input_shape=(img_h, img_w, 3))
#네트워크 동결시키기
conv_base.trainable = False
#desnet 구조 확인하기
conv_base.summary()


Model: "densenet121"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 448, 448, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d_6 (ZeroPadding2D (None, 454, 454, 3)  0           input_4[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 224, 224, 64) 9408        zero_padding2d_6[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 224, 224, 64) 256         conv1/conv[0][0]                 
________________________________________________________________________________________

In [40]:
from keras import models
from keras import layers

#헤드 부분
YOLO_model = tf.keras.models.Sequential()

initializer = tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.01, seed=None)
lrelu = tf.keras.layers.LeakyReLU(alpha=0.1)
regularizer = tf.keras.regularizers.l2(0.0005) 

YOLO_model.add(conv_base)
YOLO_model.add(tf.keras.layers.Conv2D(1024, (3, 3), activation=lrelu, kernel_initializer=initializer, kernel_regularizer = regularizer, padding = 'SAME', name = "detection_conv1", dtype='float32'))
YOLO_model.add(tf.keras.layers.Conv2D(1024, (3, 3), activation=lrelu, kernel_initializer=initializer, kernel_regularizer = regularizer, padding = 'SAME', name = "detection_conv2", dtype='float32'))
YOLO_model.add(tf.keras.layers.MaxPool2D((2, 2)))
YOLO_model.add(tf.keras.layers.Conv2D(1024, (3, 3), activation=lrelu, kernel_initializer=initializer, kernel_regularizer = regularizer, padding = 'SAME', name = "detection_conv3", dtype='float32'))
YOLO_model.add(tf.keras.layers.Conv2D(1024, (3, 3), activation=lrelu, kernel_initializer=initializer, kernel_regularizer = regularizer, padding = 'SAME', name = "detection_conv4", dtype='float32'))
# Linear 부분
YOLO_model.add(tf.keras.layers.Flatten())
YOLO_model.add(tf.keras.layers.Dense(4096, activation=lrelu, kernel_initializer = initializer, kernel_regularizer = regularizer, name = "detection_linear1", dtype='float32'))
YOLO_model.add(tf.keras.layers.Dropout(.5))
# 마지막 레이어의 활성화 함수는 선형 활성화 함수인데 이건 입력값을 그대로 내보내는거라 activation을 따로 지정하지 않았다.
YOLO_model.add(tf.keras.layers.Dense(1470, kernel_initializer = initializer, kernel_regularizer = regularizer, name = "detection_linear2", dtype='float32')) # 7*7*30 = 1470. 0~29 : (0, 0) 위치의 픽셀에 대한 각종 출력값, 30~59 : (1, 0) 위치의...블라블라
YOLO_model.add(tf.keras.layers.Reshape((7, 7, 30), name = 'output', dtype='float32'))

YOLO_model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet121 (Functional)     (None, 14, 14, 1024)      7037504   
_________________________________________________________________
detection_conv1 (Conv2D)     (None, 14, 14, 1024)      9438208   
_________________________________________________________________
detection_conv2 (Conv2D)     (None, 14, 14, 1024)      9438208   
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 7, 7, 1024)        0         
_________________________________________________________________
detection_conv3 (Conv2D)     (None, 7, 7, 1024)        9438208   
_________________________________________________________________
detection_conv4 (Conv2D)     (None, 7, 7, 1024)        9438208   
_________________________________________________________________
flatten_3 (Flatten)          (None, 50176)            

## 커스텀 학습률 스케듈러 정의하기

논문에서, 각 에포크마다 다른 학습률을 적용했기에 custom callback 함수를 사용한다.

In [41]:
from tensorflow import keras

class CustomLearningRateScheduler(keras.callbacks.Callback):


    def __init__(self, schedule):
        super(CustomLearningRateScheduler, self).__init__()
        self.schedule = schedule

        
    def on_epoch_begin(self, epoch, logs=None):
        if not hasattr(self.model.optimizer, "lr"):
            raise ValueError('Optimizer must have a "lr" attribute.')
        # 모델의 optimizer 로 부터 현재 학습률 얻기
        lr = float(tf.keras.backend.get_value(self.model.optimizer.learning_rate))
        # scheduled 된 학습률 얻기.
        scheduled_lr = self.schedule(epoch, lr)
        #에포크 시작하기 전 이번에 계산된 학습률을 optimizer에 설정해줌.
        tf.keras.backend.set_value(self.model.optimizer.lr, scheduled_lr)
        print("\nEpoch %05d: Learning rate is %6.4f." % (epoch, scheduled_lr))


LR_SCHEDULE = [
    # (에포크, 학습률)튜플
    (0, 0.01),
    (75, 0.001),
    (105, 0.0001),
]

#학습률 스케듈러 정의
def lr_schedule(epoch, lr):
    """Helper function to retrieve the scheduled learning rate based on epoch."""
    if epoch < LR_SCHEDULE[0][0] or epoch > LR_SCHEDULE[-1][0]:
        return lr
    for i in range(len(LR_SCHEDULE)):
        if epoch == LR_SCHEDULE[i][0]:
            return LR_SCHEDULE[i][1]
    return lr

## Loss Function 정의하기

Confidence = Pr*IOU 
Pr(Class|Object) -> 그리드 셀 안에 객체가 있다는 조건 하에 그 객체가 어떤 클래스인지에 대한 조건부 확률.
하나의 그리드 셀에 오직 하나의 클래스에 대한 확률 값만을 구한다.

IOU는 실제 bounding box와 예측 bounding box의 합집합면적 대 교집합면적의 비율

한 번 쭉 정리하고 코드를 보자!
Loss Function 각 줄의 내용만을 정리하면,
1. Regression of bounding box
 
 1-1) i번째 cell 에 속하는 j번째 bounding box에서 ground truth가 존재하면 -> ground truth 의 x,y 좌표 & predicted bounding box의 x,y 좌표로 regression 진행

 1-2) Regression of bounding box width and height in a cell


2. Regression of bounding box  score prediction
 
 2-1) Regression of bounding box score prediction when there is object in a cell
 
 2-2) Regression of bounding box score prediction when there isn't object in a cell
 
 
3. Class score prediction
 
 Training 이미지 데이터를 통해 -> 특정 cell 에 어떤 class 있는지 알 수 있음.
 if 학습 이미지를 입력받았을 때, P_i(C_j) = 1 #i번째 cell에서 j번째 클래스에 속해 있는

* Lambda_coord : 객체가 포함되어 있을 때 -> 학습률 5, 객체 미포함-> 학습률 0.5

In [42]:
import keras.backend as K

def xywh2minmax(xy, wh):
    xy_min = xy - wh / 2
    xy_max = xy + wh / 2

    return xy_min, xy_max

#IOU 값 정의: 실제 bounding box와 예측 bounding box의 합집합 면적 대 교집합 면적의 비율
def iou(pred_mins, pred_maxes, true_mins, true_maxes):
    #교집합 면적 최소, 최대
    intersect_mins = K.maximum(pred_mins, true_mins)
    intersect_maxes = K.minimum(pred_maxes, true_maxes)
    intersect_wh = K.maximum(intersect_maxes - intersect_mins, 0.)
    intersect_areas = intersect_wh[..., 0] * intersect_wh[..., 1]

    #예측 bounding box
    pred_wh = pred_maxes - pred_mins
    true_wh = true_maxes - true_mins
    pred_areas = pred_wh[..., 0] * pred_wh[..., 1]
    true_areas = true_wh[..., 0] * true_wh[..., 1]

    #합집합 면적
    union_areas = pred_areas + true_areas - intersect_areas
    iou_scores = intersect_areas / union_areas

    return iou_scores

#yolo_head 
def yolo_head(feats):
    # Dynamic implementation of conv dims for fully convolutional model.
    conv_dims = K.shape(feats)[1:3]  # assuming channels last
    # In YOLO the height index is the inner most iteration.
    conv_height_index = K.arange(0, stop=conv_dims[0])
    conv_width_index = K.arange(0, stop=conv_dims[1])
    conv_height_index = K.tile(conv_height_index, [conv_dims[1]])

    # TODO: Repeat_elements and tf.split doesn't support dynamic splits.
    # conv_width_index = K.repeat_elements(conv_width_index, conv_dims[1], axis=0)
    conv_width_index = K.tile(
        K.expand_dims(conv_width_index, 0), [conv_dims[0], 1])
    conv_width_index = K.flatten(K.transpose(conv_width_index))
    conv_index = K.transpose(K.stack([conv_height_index, conv_width_index]))
    conv_index = K.reshape(conv_index, [1, conv_dims[0], conv_dims[1], 1, 2])
    conv_index = K.cast(conv_index, K.dtype(feats))

    conv_dims = K.cast(K.reshape(conv_dims, [1, 1, 1, 1, 2]), K.dtype(feats))

    box_xy = (feats[..., :2] + conv_index) / conv_dims * 448
    box_wh = feats[..., 2:4] * 448

    return box_xy, box_wh


def yolo_loss(y_true, y_pred):
    label_class = y_true[..., :20]
    label_box = y_true[..., 20:24] 
    response_mask = y_true[..., 24]  
    response_mask = K.expand_dims(response_mask)  

    predict_class = y_pred[..., :20]  
    predict_trust = y_pred[..., 20:22] 
    predict_box = y_pred[..., 22:]  

    _label_box = K.reshape(label_box, [-1, 7, 7, 1, 4])
    _predict_box = K.reshape(predict_box, [-1, 7, 7, 2, 4])

    label_xy, label_wh = yolo_head(_label_box) 
    label_xy = K.expand_dims(label_xy, 3)  
    label_wh = K.expand_dims(label_wh, 3)  
    label_xy_min, label_xy_max = xywh2minmax(label_xy, label_wh)

    predict_xy, predict_wh = yolo_head(_predict_box) 
    predict_xy = K.expand_dims(predict_xy, 4) 
    predict_wh = K.expand_dims(predict_wh, 4)  
    predict_xy_min, predict_xy_max = xywh2minmax(predict_xy, predict_wh)

    iou_scores = iou(predict_xy_min, predict_xy_max, label_xy_min, label_xy_max) 
    best_ious = K.max(iou_scores, axis=4) 
    best_box = K.max(best_ious, axis=3, keepdims=True)  

    box_mask = K.cast(best_ious >= best_box, K.dtype(best_ious)) 

    no_object_loss = 0.5 * (1 - box_mask * response_mask) * K.square(0 - predict_trust)
    object_loss = box_mask * response_mask * K.square(1 - predict_trust)
    confidence_loss = no_object_loss + object_loss
    confidence_loss = K.sum(confidence_loss)

    class_loss = response_mask * K.square(label_class - predict_class)
    class_loss = K.sum(class_loss)

    _label_box = K.reshape(label_box, [-1, 7, 7, 1, 4])
    _predict_box = K.reshape(predict_box, [-1, 7, 7, 2, 4])

    label_xy, label_wh = yolo_head(_label_box)  
    predict_xy, predict_wh = yolo_head(_predict_box)  

    box_mask = K.expand_dims(box_mask)
    response_mask = K.expand_dims(response_mask)

    box_loss = 5 * box_mask * response_mask * K.square((label_xy - predict_xy) / 448)
    box_loss += 5 * box_mask * response_mask * K.square((K.sqrt(label_wh) - K.sqrt(predict_wh)) / 448)
    box_loss = K.sum(box_loss)

    loss = confidence_loss + class_loss + box_loss

    return loss


## weights 저장하기 위해 콜백 추가하기

best weights 저장하기

In [43]:
# defining a function to save the weights of best model
from tensorflow.keras.callbacks import ModelCheckpoint

mcp_save = ModelCheckpoint('weight.hdf5', save_best_only=True, monitor='val_loss', mode='min')

## 모델 컴파일하기
커스텀 loss function 사용해서 모델 컴파일

In [44]:
from tensorflow import keras

YOLO_model.compile(loss=yolo_loss ,optimizer='adam')

# 모델 훈련하기
model.fit을 불러와서 135 에포크 훈련하기

In [45]:
YOLO_model.fit(x=my_training_batch_generator,
          steps_per_epoch = int(len(train_image_dataset ) // batch_size),
          epochs = 135,
          verbose = 1,
          workers= 4,
          validation_data = my_validation_batch_generator,
          validation_steps = int(len(val_image_dataset ) // batch_size),
           callbacks=[
              CustomLearningRateScheduler(lr_schedule),
              mcp_save
          ])

Epoch 1/135

Epoch 00000: Learning rate is 0.0100.
583/625 [==========================>...] - ETA: 3:23 - loss: nan

KeyboardInterrupt: 